# Processo de limpeza dos datasets de Pleiades

### Sobre o dataset escolhido
<p>Pleiades é uma comunidade responsável por manter um "dicionário geográfico" de lugares antigos, como templos, santuários, fortes, cavernas, lagos, entre outras estruturas consideradas portadoras de valor histórico, com o objetivo de fornecer informações para pesquisas históricas, computacionais, para ferramentas de visualização e usuários curiosos sobre o assunto. </p>

<a href="https://pleiades.stoa.org/home" >Site Pleiades</a>

<p> O site construído sobre <b>o resultado final deste processo de limpeza</b> tem como público alvo curiosos sobre estruturas anciãs catalogadas pelo Pleiades e como objetivo apresentar uma exploração de seu conteúdo visando descobrir novas curiosidades sobre o que já foi encontrado.</p>

### Formatação dos dados
<p>As informações desse dicionário estão disponíveis em json e csv. Por estar com a organização mais intuitiva, os dados csv, do dia <b>08/06/2020 07:09</b>, foram selecionados. Eles estão divididos em 3 arquivos diferentes: </p>
<ul>
<li>Locations: as localizações de cada estrutura.</li>
<li>Places: as conexões entre as estruturas.</li>
<li>Names: nomes das estruturas, sua traduções literais e lingua de origem.</li>
</ul>
<p>O próprio site fornece uma explicação breve dos atributos de cada arquivo e até faz sugestão de como unificá-los. A nota de <b>06/09/2019 13:46</b> foi usada como referência para organizar e filtrar os atributos considerados relevantes para o trabalho de visualização de dados: </p>

<a href="http://atlantides.org/downloads/pleiades/dumps/README.txt" >Explicação dos atributos</a>

### Objetivo da limpeza
<p>O processo de exploração do conteúdo dos datasets, a junção entre os arquivos e a filtragem dos atributos tem como principal objetivo preparar os dados para facilitar seu uso no trabalho final da disciplina de Visualização de Dados.</p>

### PROCESSO DE LIMPEZA E GERAÇÃO DO DATASET PRINCIPAL:

#### 1) Carregando as bibliotecas e os datasets

In [1]:
import pandas as pd
import re

In [2]:
places = pd.read_csv('pleiades-places.csv')

In [3]:
names = pd.read_csv('pleiades-names.csv')

In [4]:
locations = pd.read_csv('pleiades-locations.csv')

#### 2) Observando seus formatos
<p>Locations foi o dataset escolhido como base por ter a maior quantitade de tuplas.</p> 

In [5]:
places.shape

(37156, 26)

In [6]:
names.shape

(32972, 26)

In [7]:
locations.shape

(40003, 25)

#### 3) Observando os atributos
<p><b>Atributos exclusivos encontrados em cada arquivo:</b></p>
<ul>
<li><b>Locations:</b> geometry (equivalente ao extent em places e names), featuresTypes e pid.</li>
<li><b>Places:</b> connectsWith (outros locais aos quais um se conecta), hasConnectionWith (outros locais conecatos a um).</li>
<li><b>Names:</b> nameAttested (pronúncia do nome), nameLanguage (lingua original do nome), nameTransliterated (tradução literal do nome).</li>
</ul>
<p><b>OBS: </b>Neste ponto algumas inconsistências foram encontradas entre a descrição auxiliar fornecida pelo Pleiades e o conteúdo dos csvs. Algumas colunas dadas como adicionais em uma rquivo estavam presentes nos outros dois (como featureTypes "exclusivo" de locations também presente em places e a equivalência de geometry à extent) e alguns atributos não foram encontrados (como avgRating e numRatings ausentes em names).</p>

In [8]:
places.loc[0]

authors                                          Becker, J., T. Elliott
bbox                       13.4119837, 42.082885, 13.4119837, 42.082885
connectsWith                                                     413005
created                                            2016-11-04T16:36:09Z
creators                                               jbecker, thomase
currentVersion                                                        1
description           The post-Roman settlement at Alba Fucens becam...
extent                {"type": "Point", "coordinates": [13.4119837, ...
featureTypes                                                 settlement
geoContext                                                          NaN
hasConnectionsWith                                                  NaN
id                                                             48210385
locationPrecision                                               precise
maxDate                                                         

In [9]:
locations.loc[0]

authors                                                     Becker, J.
bbox                      13.4119837, 42.082885, 13.4119837, 42.082885
created                                           2013-07-15T17:16:48Z
creators                                                       jbecker
currentVersion                                                       4
description          The post-Roman settlement at Alba Fucens. Loca...
featureType                                                settlement,
geometry             {"type": "Point", "coordinates": [13.4119837, ...
id                                        location-of-borgo-medioevale
locationPrecision                                              precise
locationType                                           representative,
maxDate                                                           1453
minDate                                                            640
modified                                          2016-11-04T23:33:08Z
path  

In [10]:
names.loc[0]

authors               Spann, P., R. Warner, R. Talbert, T. Elliott, ...
bbox                         -3.606772, 39.460299, -3.606772, 39.460299
created                                            2010-09-24T19:02:22Z
creators                                                     P.O. Spann
currentVersion                                                      NaN
description                                                         NaN
extent                {"type": "Point", "coordinates": [-3.606772, 3...
id                                                            consabura
locationPrecision                                               precise
maxDate                                                             640
minDate                                                            -330
modified                                           2011-09-05T20:57:22Z
nameAttested                                                        NaN
nameLanguage                                                    

#### 4) Comparando uma mesma tupla
<p>Após a escolha do id de um local, seus dados foram buscados e comparados nos 3 arquivos. Foi descoberto que no names um mesmo id pode ter mais de uma tupla, ou seja, um mesmo local pode ter mais de um nome, algo a ser levado em consideração e cuidado no momentode junção dos arquivos.</p>

In [11]:
places.loc[places['id'] == 265876]

,authors,bbox,connectsWith,created,creators,currentVersion,description,extent,featureTypes,geoContext,...,path,reprLat,reprLatLong,reprLong,tags,timePeriods,timePeriodsKeys,timePeriodsRange,title,uid
2,"Spann, P., DARMC, R. Talbert, S. Gillies, R. W...","-3.606772, 39.460299, -3.606772, 39.460299",NaN,2010-09-24T19:02:22Z,P.O. Spann,14.0,"An ancient settlement, likely of Celtic origin...","{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",settlement,Consuegra,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,"dare:ancient=1, dare:major=1, dare:feature=maj...",HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabura/Consabrum,3fb26862377912da0f866fc310bcaf0c


In [12]:
locations.loc[locations['pid'] == '/places/265876']

,authors,bbox,created,creators,currentVersion,description,featureType,geometry,id,locationPrecision,...,pid,reprLat,reprLatLong,reprLong,tags,timePeriods,timePeriodsKeys,timePeriodsRange,title,uid
2,"Spann, P., D. R. Talbert, T. Elliott, S. Gillies","-3.606772, 39.460299, -3.606772, 39.460299",2011-03-10T00:05:52Z,P.O. Spann,1.0,1M scale point location,unknown,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",darmc-location-20192,precise,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",DARMC location 20192,62e35ae520c733fd1b3d538cfc93024d


In [13]:
names.loc[names['pid'] == '/places/265876']

,authors,bbox,created,creators,currentVersion,description,extent,id,locationPrecision,maxDate,...,pid,reprLat,reprLatLong,reprLong,tags,timePeriods,timePeriodsKeys,timePeriodsRange,title,uid
0,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",consabura,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabura,fc0ee157ff11ce6d2e72cd7c5df06fee
1,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",consabrum,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabrum,e2b8756302fb62ff0e301710c265a3e6
2,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",kondabora,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Kondabora,f742b67b8343a6c68f222fbf607dcbf9
3,"Becker, J.","-4.0, 39.0, -3.0, 40.0",2016-02-08T23:21:01Z,jbecker,0.0,An ethnic name used by Pliny the Elder.,"{""type"": ""Polygon"", ""coordinates"": [[[-4.0, 39...",consaburrenses,precise,300.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,R,roman,"-30.0,300.0",Consaburrenses,ddc6939eba8aaf3729e6c254dedc90e4


#### 5) Selecionando atributos e removendo os desnecessários
<p>Os seguintes atributos de cada arquivo foram considerados relevantes para a visualização do trabalho:</p>
<ul>
<li><b>Locations:</b> bbox, description, featureType, geometry, maxDate, minDate, pid, reprLat, reprLong, tags, timePeriodsKeys, title.</li>
<li><b>Places:</b> connectsWith, hasConnectionsWith.</li>
<li><b>Names:</b> nameAttested, nameLanguage, nameTransliterated.</li>
</ul>
<p><b>OBS: </b>Atributos não listados foram descatados por não serem relevantes (como autor, data de alteração da tupla, etc) ou por serem redundantes (como reprLatLong ser a junção de reprLat com reprLong).</p>

In [14]:
locations = locations.filter(items=['pid', 'title','featureType','reprLat','reprLong','bbox',
                                    'geometry','description','maxDate','minDate','tags','timePeriodsKeys'])

In [15]:
locations.loc[0]

pid                                                 /places/48210385
title                                OSM location of Borgo Medievale
featureType                                              settlement,
reprLat                                                      42.0829
reprLong                                                      13.412
bbox                    13.4119837, 42.082885, 13.4119837, 42.082885
geometry           {"type": "Point", "coordinates": [13.4119837, ...
description        The post-Roman settlement at Alba Fucens. Loca...
maxDate                                                         1453
minDate                                                          640
tags                                                  extant remains
timePeriodsKeys                                  mediaeval-byzantine
Name: 0, dtype: object

In [16]:
places = places.filter(items=['id','connectsWith','hasConnectionsWith'])

In [17]:
places.loc[0]

id                    48210385
connectsWith            413005
hasConnectionsWith         NaN
Name: 0, dtype: object

In [18]:
names = names.filter(items=['pid','nameAttested','nameLanguage','nameTransliterated'])

In [19]:
names.loc[0]

pid                   /places/265876
nameAttested                     NaN
nameLanguage                     NaN
nameTransliterated         Consabura
Name: 0, dtype: object

#### 6) Adaptando alguns tipos
<p>Alguns atributos, como tags e featureTypes, foram re-organizados para facilitar sua leitura na visualização. Além disso, algumas tags e ids estavam repetidas dentro de uma mesma célula de um mesmo atributo, estas repetições também foram descartadas, e alguns dados com espaços e caracteres desnecessários, ou que poderiam atrapalhar na visualização posteriormente, foram limpos com uso de regular expressions.</p>
<p><b>OBS: </b>Foi observado durante a adaptação dos dados de locations que existem locais redundantes, estes serão tratados posteriormente caso a caso.</p>

In [20]:
def splitLocationAttrs(dataset):
    for i in range(len(dataset.index)):
        
        idd = int(re.search(r'\d+', str(dataset.at[i, 'pid'])).group(0))
        dataset.at[i, 'pid'] = idd
        
        ################################
        
        box = str(dataset.at[i, 'bbox'])
        if(box == "nan"):
            dataset.at[i, 'bbox'] = []
        else:
            box = box.split(",")
            if(box[-1] == ''):
                box = box[:-1]
            for w in range(len(box)):
                box[w] = re.sub(r'^\s{1,}', '', box[w])
            box = [float(x) for x in box]
            dataset.at[i, 'bbox'] = box
        
        ################################
        
        feature = str(dataset.at[i, 'featureType'])
        if(feature == "nan"):
            dataset.at[i, 'featureType'] = ['unknown']
        else:
            feature = feature.split(",")
            if(feature[-1] == ''):
                feature = feature[:-1]
            for w in range(len(feature)):
                feature[w] = re.sub(r'^\s{1,}', '', feature[w])
            feature = list(set(feature))
            dataset.at[i, 'featureType'] = feature
        
        ################################
        
        tag = str(dataset.at[i, 'tags'])
        if(tag == "nan"):
            dataset.at[i, 'tags'] = []
        else:
            tag = tag.split(",")
            if(tag[-1] == ''):
                tag = tag[:-1]
            for w in range(len(tag)):
                tag[w] = re.sub(r'^\s{1,}', '', tag[w])
            tag = list(set(tag))
            dataset.at[i, 'tags'] = tag        
        
        ################################
        
        pkeys = str(dataset.at[i, 'timePeriodsKeys'])  
        if(pkeys == "nan"):
            dataset.at[i, 'timePeriodsKeys'] = ['unknown']
        else:
            pkeys = pkeys.split(",")
            if(pkeys[-1] == ''):
                pkeys = pkeys[:-1]
            for w in range(len(pkeys)):
                pkeys[w] = re.sub(r'^\s{1,}', '', pkeys[w])
            pkeys = list(set(pkeys))
            dataset.at[i, 'timePeriodsKeys'] = pkeys

In [21]:
splitLocationAttrs(locations)

In [22]:
locations.loc[0]

pid                                                         48210385
title                                OSM location of Borgo Medievale
featureType                                             [settlement]
reprLat                                                      42.0829
reprLong                                                      13.412
bbox                  [13.4119837, 42.082885, 13.4119837, 42.082885]
geometry           {"type": "Point", "coordinates": [13.4119837, ...
description        The post-Roman settlement at Alba Fucens. Loca...
maxDate                                                         1453
minDate                                                          640
tags                                                [extant remains]
timePeriodsKeys                                [mediaeval-byzantine]
Name: 0, dtype: object

In [23]:
def splitPlaceAttrs(dataset):
    for i in range(len(dataset.index)):        
        
        cw = str(dataset.at[i, 'connectsWith'])
        if(cw == "nan"):
            dataset.at[i, 'connectsWith'] = []
        else:
            cw = cw.split(",")
            if(cw[-1] == ''):
                cw = cw[:-1]
            for w in range(len(cw)):
                cw[w] = re.search(r'\d+', str(cw)).group(0)
            cw = list(set([int(x) for x in cw]))
            dataset.at[i, 'connectsWith'] = cw
        
        ################################
        
        hc = str(dataset.at[i, 'hasConnectionsWith'])
        if(hc == "nan"):
            dataset.at[i, 'hasConnectionsWith'] = []
        else:
            hc = hc.split(",")
            if(hc[-1] == ''):
                hc = hc[:-1]
            for w in range(len(hc)):
                hc[w] = re.search(r'\d+', str(hc)).group(0)
            hc = list(set([int(x) for x in hc]))
            dataset.at[i, 'hasConnectionsWith'] = hc

In [24]:
splitPlaceAttrs(places)

In [25]:
places.loc[0]

id                    48210385
connectsWith          [413005]
hasConnectionsWith          []
Name: 0, dtype: object

In [26]:
names = pd.read_csv('pleiades-names.csv')

In [27]:
names = names.filter(items=['pid','nameAttested','nameLanguage','nameTransliterated'])

In [28]:
def splitNamesAttrs(dataset):
    for i in range(len(dataset.index)):
        
        idd = int(re.search(r'\d+', str(dataset.at[i, 'pid'])).group(0))
        dataset.at[i, 'pid'] = idd
        
        ################################
        
        na = str(dataset.at[i, 'nameAttested'])
        if(na == "nan"):
            dataset.at[i, 'nameAttested'] = 'unknown'
        else:
            na = re.sub(r'^\s{1,}', '', na)
            dataset.at[i, 'nameAttested'] = na
        
        ################################
        
        nl = str(dataset.at[i, 'nameLanguage'])
        if(nl== "nan"):
            dataset.at[i, 'nameLanguage'] = 'unknown'
        else:
            nl = re.sub(r'^\s{1,}', '', nl)
            dataset.at[i, 'nameLanguage'] = nl
        
        ################################
        
        nt = str(dataset.at[i, 'nameTransliterated'])
        if(nt == "nan"):
            dataset.at[i, 'nameTransliterated'] = ['unknown']
        else:
            nt = nt.split(",")
            if(nt[-1] == ''):
                nt = nt[:-1]
            for w in range(len(nt)):
                nt[w] = re.sub(r'^\s{1,}', '', nt[w])
            nt = list(set(nt))
            dataset.at[i, 'nameTransliterated'] = nt

In [29]:
splitNamesAttrs(names)

In [30]:
names.loc[0]

pid                        265876
nameAttested              unknown
nameLanguage              unknown
nameTransliterated    [Consabura]
Name: 0, dtype: object

#### 7) Filtrando identificadores repetidos
<p>Como em fases anteriores foram descobertas redundâncias, muito além das esperadas, cada caso foi avaliado e os atributos combinados para se ter a informação mais completa possível. No caso, MUITAS tuplas são repetidas, portanto, um arquivo auxiliar foi gerado com as limpezas que foram realizadas até o momento.</p>
<p><b>OBS: </b>Os arquivos intermediários foram usados para fazer o <b>protótipo</b> das visualizações.</p>

#### a) Savepoint :D

In [34]:
locations.to_csv('LocationsPV.csv', index=False)

In [35]:
places.to_csv('PlacesPV.csv', index=False)

In [36]:
names.to_csv('NamesPV.csv', index=False)

#### b) Clean for the Win

In [429]:
dd = locations.pivot_table(index=['pid'], aggfunc='size')

for x in range(len(dd)):
    if(dd.iloc[x] > 1):
        print(dd.index[x])

20430
20484
20531
29457
29620
29625
29755
39281
39376
39410
39439
39441
49903
50076
50118
50156
50157
50252
59669
59740
59822
59851
59905
59968
59969
60056
60088
60476
69463
69468
69471
69491
69494
69507
69515
69518
69535
69538
69539
69540
69579
69584
79285
79287
79291
79293
79299
79313
79331
79335
79346
79350
79352
79353
79362
79363
79368
79370
79371
79377
79378
79383
79387
79389
79393
79395
79407
79418
79419
79420
79431
79432
79433
79434
79435
79440
79460
79475
79484
79488
79492
79502
79509
79532
79542
79549
79552
79556
79559
79562
79565
79569
79583
79587
79590
79592
79595
79602
79616
79622
79631
79645
79651
79654
79656
79664
79677
79684
79688
79705
79706
79708
79722
79727
79733
79734
79735
79741
79768
79777
89092
89095
89104
89109
89113
89117
89122
89124
89128
89133
89136
89138
89144
89145
89148
89150
89155
89182
89193
89201
89206
89224
89234
89236
89238
89239
89243
89251
89274
89287
89288
89307
89309
89311
89313
89314
89315
89321
92147
98924
98925
98932
98933
98934
98956
98970
9897

442439
442451
442456
442471
442490
442493
442507
442509
442515
442525
442543
442545
442550
442566
442567
442570
442572
442599
442603
442615
442628
442629
442632
442640
442642
442656
442691
442720
442733
442735
442744
442754
442758
442785
442790
442796
442797
442804
442834
442839
442847
442851
442855
443958
452255
452261
452281
452317
452333
452352
452369
452377
452416
452435
452438
452439
452449
452454
452457
452488
456108
462062
462067
462068
462069
462073
462077
462081
462082
462097
462106
462109
462111
462114
462120
462123
462126
462130
462132
462135
462151
462154
462176
462179
462203
462204
462214
462225
462228
462234
462237
462240
462258
462263
462269
462281
462283
462290
462304
462307
462317
462354
462358
462365
462373
462381
462386
462388
462406
462416
462423
462424
462432
462457
462459
462466
462470
462481
462487
462498
462506
462511
462513
462518
462520
462527
462532
465881
465884
471909
471979
472048
472053
472079
472085
481728
481741
481777
481782
481783
481784
481808
481818

874442
874444
874492
874502
874545
874572
874573
874593
874621
874623
874662
874699
874730
874754
874763
874774
874806
875049
884109
884148
884193
884201
884905
893922
893928
893943
893976
893990
894019
894028
894084
894089
894132
894158
894179
894183
894184
894185
897862
902990
903021
903102
903104
903126
912798
912871
912872
912873
912903
912975
922617
922618
922627
922675
922685
922693
922695
922704
922706
922711
922714
922729
922740
942248
942276
942281
952059
952108
961873
961875
961886
961971
971695
971701
971739
971746
971748
971757
971866
971878
971894
971899
971910
2970016
3444300
3445460
17649950
58129442
59320269
103808101
104811110
108542249
122572946
122702818
127640903
136699833
142847211
155598151
165170193
197001409
200345143
202833933
208942158
208942846
214908564
264120821
266705016
271500627
272860363
298394661
303708957
307187361
321318358
324841375
324841376
324841377
324841378
324841379
326150788
336410435
344182764
344182765
345438602
354354474
356648400
35756583

In [424]:
locations.loc[locations['pid'] == 20430]

,pid,title,featureType,reprLat,reprLong,bbox,geometry,description,maxDate,minDate,tags,timePeriodsKeys
29468,20430,Barrington Atlas location,[unknown],57.5000,22.500000,"[20.0, 55.0, 25.0, 60.0]","{""type"": ""Polygon"", ""coordinates"": [[[20.0, 55...",NaN,300.0,-30.0,[],[roman]
29469,20430,OSM location of Daugava (river),[river],56.8635,24.255967,"[24.2559671, 56.8635004, 24.2559671, 56.8635004]","{""type"": ""Point"", ""coordinates"": [24.2559671, ...",Representative location based on OpenStreetMap.,NaN,NaN,[],[unknown]


In [425]:
locations.loc[locations['pid'] == 20484]

,pid,title,featureType,reprLat,reprLong,bbox,geometry,description,maxDate,minDate,tags,timePeriodsKeys
29495,20484,DARMC location 12694,[unknown],59.278903,17.676204,"[17.676204, 59.278903, 17.676204, 59.278903]","{""type"": ""Point"", ""coordinates"": [17.676204, 5...",5M scale point location,640.0,300.0,[],[late-antique]
29496,20484,DARE Location,[settlement],59.278511,17.680435,"[17.680435, 59.278511, 17.680435, 59.278511]","{""type"": ""Point"", ""coordinates"": [17.680435, 5...","Representative point location, site precision",640.0,300.0,[],[late-antique]


In [426]:
locations.loc[locations['pid'] == 989985175]

,pid,title,featureType,reprLat,reprLong,bbox,geometry,description,maxDate,minDate,tags,timePeriodsKeys
25022,989985175,OSM location of Roman amphitheater,[amphitheatre],53.189223,-2.887809,"[-2.8878093, 53.1892233, -2.8878093, 53.1892233]","{""type"": ""Point"", ""coordinates"": [-2.8878093, ...",The Roman amphitheater at Deva was the largest...,300.0,-30.0,[],[roman]
25023,989985175,Imagery location of Centerpoint of amphitheatre,[amphitheatre],53.189167,-2.886944,"[-2.886944, 53.189167, -2.886944, 53.189167]","{""type"": ""Point"", ""coordinates"": [-2.886944, 5...","Coordinates following GeoHack, verified in Goo...",300.0,-30.0,[],[roman]


In [427]:
locations.loc[locations['pid'] == 987039111]

,pid,title,featureType,reprLat,reprLong,bbox,geometry,description,maxDate,minDate,tags,timePeriodsKeys
22698,987039111,Imagery location of the Mausoleum of Helena,[tomb],41.878889,12.548611,"[12.548611, 41.878889, 12.548611, 41.878889]","{""type"": ""Point"", ""coordinates"": [12.548611, 4...",verified in Google Earth 2013,640.0,300.0,[],[late-antique]
22699,987039111,OSM location of Mausoleum of Helena,[tomb],41.878889,12.548815,"[12.5486384, 41.878777, 12.5489817, 41.8790286]","{""type"": ""Polygon"", ""coordinates"": [[[12.54876...",Representative location based on OpenStreetMap.,640.0,300.0,[],[late-antique]


#### 8) Combinando location e places
<p>O id em places foi mantido temporariamente para que fosse possível combinar location com seus atributos relevantes. Apenas por precaução, tanto location quanto places terão possíveis ids duplicados descartados, não há necessidade de fazer o mesmo para names, já que é considerado normal para este arquivo.</p>